In [1]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from urllib.parse import unquote

server_name   = "DESKTOP-H94U7MR\MSSQLSERVER01"
database_name = "bokhandeln"

connection_string = f"DRIVER=ODBC Driver 17 for SQL Server;SERVER={server_name};DATABASE={database_name};Trusted_Connection=yes"
url_string        = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

print('Connecting to database using URL string:')
unquoted_url = unquote(str(url_string))
print(unquoted_url, '\n')

try:    
    engine = create_engine(url_string)
    with engine.connect() as connection:
        print(f'Successfully connected to {database_name}!')
except Exception as e:
    print('Error while connecting to database:\n')
    print(e)

Connecting to database using URL string:
mssql+pyodbc://?odbc_connect=DRIVER=ODBC+Driver+17+for+SQL+Server;SERVER=DESKTOP-H94U7MR\MSSQLSERVER01;DATABASE=bokhandeln;Trusted_Connection=yes 

Successfully connected to bokhandeln!


In [4]:
from sqlalchemy import MetaData, Table, Column, Integer, String, ForeignKey
metadata_obj = MetaData()

Book_table = Table(
    "Böcker",
    metadata_obj,
    Column("ISBN13", Integer, primary_key=True),
    Column("Titel", String(30)),
    Column("Språk", String),
    Column("Utgivningsdatum", String),
	Column("Genre", String),
	Column("FörfattarID", Integer),
	Column("Pris", Integer)
)

Store_table = Table(
    "Butiker",
    metadata_obj,
    Column("ButikID", Integer, primary_key=True),
    Column("Butiksnamn", String(30)),
    Column("Adress", String),
    Column("Kontaktinformation", String)
)

Author_table = Table(
    "Författare",
    metadata_obj,
    Column("FörfattarID", Integer, primary_key=True),
    Column("Förnamn", String(30)),
    Column("Efternamn", String),
    Column("Födelsedatum", String)
)

category_table = Table(
    "Kategorier",
    metadata_obj,
    Column("KategoriID", Integer),
    Column("Kategorinamn", String(30))
)

Inventory_table = Table(
    "LagerSaldo",
    metadata_obj,
    Column("ButikID", Integer, primary_key=True),
    Column("ISBN", Integer),
    Column("Exemplar", String)
)

Order_det_table = Table(
    "Orderdetaljer",
    metadata_obj,
    Column("OrderdetailID", Integer, primary_key=True),
    Column("OrderID", Integer),
    Column("ISBN13", Integer),
    Column("Antal", Integer),
    Column("Pris", Integer)
)

Customers_table = Table(
    "Kunder",
    metadata_obj,
    Column("KundID", Integer, primary_key=True),
    Column("Namn", String),
    Column("Adress", String),
    Column("Telefonnummer", Integer)
)

Orders_table = Table(
    "Beställningar",
    metadata_obj,
    Column("KundOrderID", Integer, primary_key=True),
    Column("KundID", Integer),
    Column("Beställningsdatum", String),
    Column("Status", String)
)

for name, table in metadata_obj.tables.items():
    print(f"Table: {name}")

    for column in table.c:
        print(f"{column.name.ljust(15)}{column.type}")

    print()

Table: Böcker
ISBN13         INTEGER
Titel          VARCHAR(30)
Språk          VARCHAR
UtgivningsdatumVARCHAR
Genre          VARCHAR
FörfattarID    INTEGER
Pris           INTEGER

Table: Butiker
ButikID        INTEGER
Butiksnamn     VARCHAR(30)
Adress         VARCHAR
KontaktinformationVARCHAR

Table: Författare
FörfattarID    INTEGER
Förnamn        VARCHAR(30)
Efternamn      VARCHAR
Födelsedatum   VARCHAR

Table: Kategorier
KategoriID     INTEGER
Kategorinamn   VARCHAR(30)

Table: LagerSaldo
ButikID        INTEGER
ISBN           INTEGER
Exemplar       VARCHAR

Table: Orderdetaljer
OrderdetailID  INTEGER
OrderID        INTEGER
ISBN13         INTEGER
Antal          INTEGER
Pris           INTEGER

Table: Kunder
KundID         INTEGER
Namn           VARCHAR
Adress         VARCHAR
Telefonnummer  INTEGER

Table: Beställningar
KundOrderID    INTEGER
KundID         INTEGER
BeställningsdatumVARCHAR
Status         VARCHAR



In [11]:
from sqlalchemy import text

name = 'Ondskan'

parameterized_query = text("SELECT * FROM Böcker WHERE Titel LIKE '%' + :Search_title + '%'")

with engine.connect() as conn:    
    result = conn.execute(parameterized_query, {"Search_title": name})
    print(result.fetchall())

[('9789164230058', 'Ondskan', 'Svenska', '2004-12-10', 'Samtida skönlitteratur', 8, Decimal('299.99'))]
